In [1]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *

import plotly.express as px
from ipywidgets import interactive

#%load_ext watermark
#!jupyter serverextension enable voila --sys-prefix
#%watermark --iversions --python

In [2]:
print ('Whilst you are waiting impatiently to be allowed out, why not check the brand of your running shoes as a distraction from the Covid-19 figures. When Spain\'s curve flattens enough you can put your shoes on again and go for a run!')
print ('\nUpload a photo of your shoes and check whether they are by Adidas, Nike or Asics.')

Whilst you are waiting imapatiently to be allowed out, why not check the brand of your running shoes as a distraction from th Covid-19 figures. When Spain's curve flattens enough you can put your shoes on again and go for a run!

Upload a photo of your shoes and check whether they are by Adidas, Nike or Asics.


In [3]:
path = Path()
learn_inf = load_learner(path/'export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [4]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Predicted brand: {pred}, with probability: {probs[pred_idx]:.04f}'

In [5]:
btn_run.on_click(on_click_classify)
#btn_upload.observe(on_click_classify, names=['data'])

In [11]:
VBox([widgets.Label('Find a picture of your running shoe:'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [12]:
print ('Then check out the latest plot to see when we may be allowed out. One day more...')

Then check the latest plot to see when we may be allowed out. One day more...


In [8]:
df = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/full_data.csv', index_col='date', parse_dates=True)
data = df[df.location.isin(['Spain','United States'])]
data=data[data.index>'2020-02-29']

In [9]:
#https://voila-gallery.org/
#https://github.com/mkcor/covid-plots/blob/master/covid_it_fr_lag.ipynb

In [10]:
def plot_compare_lag(variable, lag, window):
    wide_data = data.pivot(columns='location', values=variable)
    wide_data['USA lag'] = wide_data['United States'].shift(-lag)
    wide_data = wide_data[['USA lag', 'Spain']].rolling(window=window).mean()
    wide_data.reset_index(level=0, inplace=True)
    fig = px.line(wide_data.melt(id_vars='date'), x='date', y='value', color='location')
    fig.update_layout(
    title='Covid-19 Comparative: Lag of '+str(lag)+' days, '+str(window)+'-day rolling average',
    xaxis_title="Spain date",
    yaxis_title="Log scale",
    yaxis_type="log")
    fig.show()

interactive(plot_compare_lag, variable=list(data.columns[1:]), lag=(0, 8), window=(1,6))

interactive(children=(Dropdown(description='variable', options=('new_cases', 'new_deaths', 'total_cases', 'tot…